In [ ]:
# =========================
# Dependências e setup
# =========================
from google.colab import drive
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
import re
import spacy
import nltk
from nltk.corpus import wordnet as wn
from datetime import datetime

nltk.download('wordnet')
nltk.download('omw-1.4')

!pip install -q spacy openpyxl
!python -m spacy download pt_core_news_lg

nlp = spacy.load("pt_core_news_lg")

In [ ]:
# ======================
# Montar Google Drive
# ======================
drive.mount('/content/drive/')
file = "/content/drive/MyDrive/Constructions_concordances/EXEMPLO.xml" #<------ Substituir nome do ficheiro pelo do ficheiro a analisar

In [ ]:
# ======================
# Escolher a construção
# ======================
tipo_construcao = "svo"  # 'svo', 'n_adj', 'adj_n'

In [ ]:
# =============================
# Funções de pré-processamento
# =============================
def limpar_kwic(texto_kwic):
    return re.sub(r"/[a-z]+", "", texto_kwic)

with open(file, "r", encoding="utf-8") as f:
    xml = f.read()

soup = BeautifulSoup(xml, "xml")
kwics = soup.find_all("kwic")
kwic_pairs = [(limpar_kwic(k.text.strip()), k.text.strip()) for k in kwics]

frases_limpas = [pair[0] for pair in kwic_pairs]
frases_originais = [pair[1] for pair in kwic_pairs]

In [ ]:
# ===================================
# Funções de extração sintática
# ===================================
def extrair_svo(frase):
    doc = nlp(frase)
    sujeito = verbo = objeto = None
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            verbo = token.lemma_
        elif token.dep_ == "nsubj":
            sujeito = token.text
        elif token.dep_ in {"obj", "dobj", "obl", "attr"}:
            objeto = token.lemma_
    if verbo and (sujeito or objeto):
        return {
            "frase_limpa": frase,
            "sujeito": sujeito if sujeito else "",
            "verbo": verbo,
            "objeto": objeto if objeto else ""
        }
    return None

def extrair_n_adj(frase):
    doc = nlp(frase)
    for token in doc:
        if token.pos_ == "ADJ" and token.head.pos_ == "NOUN":
            nome = token.head.lemma_
            adj = token.lemma_
            return {"frase_limpa": frase, "nome": nome, "adjetivo": adj}
    return None

def extrair_adj_n(frase):
    doc = nlp(frase)
    for token in doc:
        if token.pos_ == "ADJ" and token.head.pos_ == "NOUN" and token.i < token.head.i:
            return {"frase_limpa": frase, "adjetivo": token.lemma_, "nome": token.head.lemma_}
    for i in range(len(doc) - 1):
        if doc[i].pos_ == "ADJ" and doc[i + 1].pos_ == "NOUN":
            return {"frase_limpa": frase, "adjetivo": doc[i].lemma_, "nome": doc[i + 1].lemma_}
    return None


In [ ]:
# ===============================
# Domínios WordNet + Mapeamento
# ===============================
mapeamento_dominios = {
    'noun.person': 'pessoa',
    'noun.artifact': 'objeto',
    'noun.act': 'evento',
    'noun.event': 'evento',
    'noun.group': 'organização',
    'noun.location': 'lugar',
    'noun.communication': 'comunicação',
    'noun.state': 'estado',
    'noun.cognition': 'conhecimento',
    'noun.quantity': 'quantidade',
    'noun.attribute': 'característica',
    'noun.time': 'tempo',
    'noun.animal': 'animal',
    'noun.body': 'corpo',
    'noun.food': 'comida',
    'noun.substance': 'matéria',
    'noun.object': 'objeto',
    'noun.feeling': 'emoção',
    'noun.phenomenon': 'fenómeno',
}

def obter_dominios(word, lang='por'):
    synsets = wn.synsets(word, lang=lang)
    if not synsets:
        return "desconhecido", "desconhecido"
    primeiro = synsets[0]
    subdominio = primeiro.lexname()
    dominio_mapeado = mapeamento_dominios.get(subdominio, "outro")
    return dominio_mapeado, subdominio


In [ ]:
# ========================================
# Processamento das frases e domínios
# ========================================
dados = []
for frase_limpa, frase_original in kwic_pairs:
    if tipo_construcao == "svo":
        extraido = extrair_svo(frase_limpa)
    elif tipo_construcao == "n_adj":
        extraido = extrair_n_adj(frase_limpa)
    elif tipo_construcao == "adj_n":
        extraido = extrair_adj_n(frase_limpa)
    else:
        extraido = None
    if extraido:
        extraido["frase_original"] = frase_original
        extraido["frase_limpa"] = frase_limpa
        dados.append(extraido)

print(f"Número de frases extraídas: {len(dados)}")

for entrada in dados:
    if tipo_construcao == "svo":
        entrada["dominio"], entrada["subdominio"] = obter_dominios(entrada["objeto"])
        entrada["construcao"] = f"{entrada['verbo']} X"
    elif tipo_construcao == "n_adj":
        entrada["dominio"], entrada["subdominio"] = obter_dominios(entrada["nome"])
        entrada["construcao"] = f"{entrada['nome']} + {entrada['adjetivo']}"
    elif tipo_construcao == "adj_n":
        entrada["dominio"], entrada["subdominio"] = obter_dominios(entrada["nome"])
        entrada["construcao"] = f"{entrada['adjetivo']} {entrada['nome']}"

df = pd.DataFrame(dados)

# ✅ NORMALIZAÇÃO
if tipo_construcao in ["n_adj", "adj_n"]:
    df["nome"] = df["nome"].str.lower()
    df["adjetivo"] = df["adjetivo"].str.lower()

if tipo_construcao == "svo":
    verbos_leves = {"fazer", "ter", "dar", "estar", "haver", "ficar", "pôr", "levar", "deixar", "manter"}
    df = df[~df["verbo"].isin(verbos_leves)].copy()


In [ ]:
# ==========================================
# Análise de variabilidade semântica
# ==========================================
if tipo_construcao == "svo":
    # Variabilidade verbo → objeto
    agrupados_verbo_obj = df.groupby("verbo")["dominio"].apply(list)
    df_var_verbo_obj = agrupados_verbo_obj.apply(lambda x: pd.Series({
        "variabilidade_verbo_obj": len(set(x)),
        "dominios_obj": ", ".join(sorted(set(x)))
    })).reset_index()

    # Variabilidade verbo → sujeito
    df['dominio_sujeito'], df['subdominio_sujeito'] = zip(*df['sujeito'].apply(obter_dominios) if "sujeito" in df else [("desconhecido", "desconhecido")] * len(df))
    agrupados_verbo_suj = df.groupby("verbo")["dominio_sujeito"].apply(list)
    df_var_verbo_suj = agrupados_verbo_suj.apply(lambda x: pd.Series({
        "variabilidade_verbo_suj": len(set(x)),
        "dominios_suj": ", ".join(sorted(set(x)))
    })).reset_index()


elif tipo_construcao == "adj_n":
    agrupados = df.groupby("adjetivo")["dominio"].apply(list)
    df_var = agrupados.apply(lambda x: pd.Series({
        "variabilidade_semântica": len(set(x)),
        "dominios": ", ".join(sorted(set(x)))
    })).reset_index()
    df_var = df_var.rename(columns={"adjetivo": "construcao"})

elif tipo_construcao == "n_adj":
    df['dominio_adjetivo'], df['subdominio_adjetivo'] = zip(*df['adjetivo'].apply(obter_dominios))

    agrupados_nome = df.groupby("nome")["dominio_adjetivo"].apply(list)
    df_var_nome = agrupados_nome.apply(lambda x: pd.Series({
        "variabilidade_nome": len(set(x)),
        "dominios": ", ".join(sorted(set(x)))
    })).reset_index()

    agrupados_adj = df.groupby("adjetivo")["dominio"].apply(list)
    df_var_adj = agrupados_adj.apply(lambda x: pd.Series({
        "variabilidade_adjetivo": len(set(x)),
        "dominios": ", ".join(sorted(set(x)))
    })).reset_index()



In [ ]:
# ===============================
# Exibir exemplos e estatísticas
# ===============================
print(f"\n📊 Top 10 construções ({tipo_construcao}):")

if tipo_construcao == "n_adj":
    df_var_nome_sorted = df_var_nome.sort_values(by="variabilidade_nome", ascending=False)
    df_var_adj_sorted = df_var_adj.sort_values(by="variabilidade_adjetivo", ascending=False)

    print("\n🔸 Variabilidade por **nome**:")
    print(df_var_nome_sorted.head(10))

    print("\n🔸 Variabilidade por **adjetivo**:")
    print(df_var_adj_sorted.head(10))

    print(f"\n📌 Exemplos para os 5 nomes mais variáveis:")
    for nome in df_var_nome_sorted.head(5)["nome"]:
        print(f"\n🔹 Nome: {nome.capitalize()}")
        frases_filtradas = df[df["nome"] == nome].head(4)
        for _, row in frases_filtradas.iterrows():
            print(f" - Frase: {row['frase_limpa']} | Adjetivo: {row['adjetivo']} | Domínio: {row['dominio_adjetivo']} | Subdomínio: {row['subdominio_adjetivo']}")

    print(f"\n📌 Exemplos para os 5 adjetivos mais variáveis:")
    for adj in df_var_adj_sorted.head(5)["adjetivo"]:
        print(f"\n🔹 Adjetivo: {adj}")
        frases_filtradas = df[df["adjetivo"] == adj].head(4)
        for _, row in frases_filtradas.iterrows():
            print(f" - Frase: {row['frase_limpa']} | Nome: {row['nome']} | Domínio: {row['dominio']} | Subdomínio: {row['subdominio']}")

elif tipo_construcao == "svo":
    df_var_obj_sorted = df_var_verbo_obj.sort_values(by="variabilidade_verbo_obj", ascending=False)
    df_var_suj_sorted = df_var_verbo_suj.sort_values(by="variabilidade_verbo_suj", ascending=False)

    print("\n🔸 Variabilidade verbo → objeto:")
    print(df_var_obj_sorted.head(10))

    print("\n🔸 Variabilidade verbo → sujeito:")
    print(df_var_suj_sorted.head(10))

    print(f"\n📌 Exemplos para os 5 verbos mais variáveis (objeto):")
    for verbo in df_var_obj_sorted.head(5)["verbo"]:
        print(f"\n🔹 Verbo: {verbo}")
        frases_filtradas = df[df["verbo"] == verbo].head(4)
        for _, row in frases_filtradas.iterrows():
            print(f" - Frase: {row['frase_limpa']} | Objeto: {row['objeto']} | Domínio_obj: {row['dominio']} | Subdomínio_obj: {row['subdominio']}")

    print(f"\n📌 Exemplos para os 5 verbos mais variáveis (sujeito):")
    for verbo in df_var_suj_sorted.head(5)["verbo"]:
        print(f"\n🔹 Verbo: {verbo}")
        frases_filtradas = df[df["verbo"] == verbo].head(4)
        for _, row in frases_filtradas.iterrows():
            print(f" - Frase: {row['frase_limpa']} | Sujeito: {row['sujeito']} | Domínio_suj: {row['dominio_sujeito']} | Subdomínio_suj: {row['subdominio_sujeito']}")


elif tipo_construcao == "adj_n":
    df_var_sorted = df_var.sort_values(by="variabilidade_semântica", ascending=False)

    print(df_var_sorted.head(10))
    print(f"\n📌 Exemplos para os 5 adjetivos mais variáveis:")
    for adj in df_var_sorted.head(5)["construcao"].str.split().str[0]:
        print(f"\n🔹 Adjetivo: {adj}")
        frases_filtradas = df[df["adjetivo"] == adj].head(4)
        for _, row in frases_filtradas.iterrows():
            print(f" - Frase: {row['frase_limpa']} | Nome: {row['nome']} | Domínio: {row['dominio']} | Subdomínio: {row['subdominio']}")



In [ ]:
# ===============================
# Estatísticas gerais
# ===============================
total = len(df)
desconhecidos = df[df['dominio'] == "desconhecido"].shape[0]
percentagem = 100 * (total - desconhecidos) / total if total else 0
print(f"\n📊 Percentagem de Domínios Atribuídos:")
print(f" - Atribuídos: {percentagem:.2f}%")
print(f" - 'Desconhecido': {100 - percentagem:.2f}%")

In [ ]:
# ===============================
# Exportar para Excel na Drive
# ===============================

# Criar nome do ficheiro com tipo da construção e data/hora
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"/content/drive/MyDrive/Constructions_concordances/output_variabilidade_{tipo_construcao}_{timestamp}.xlsx"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    df.to_excel(writer, index=False, sheet_name="Construcoes")

    if tipo_construcao == "svo":
        df_var_obj_sorted = df_var_verbo_obj.sort_values(by="variabilidade_verbo_obj", ascending=False)
        df_var_suj_sorted = df_var_verbo_suj.sort_values(by="variabilidade_verbo_suj", ascending=False)
        df_var_obj_sorted.to_excel(writer, index=False, sheet_name="Variabilidade_verbo_objeto")
        df_var_suj_sorted.to_excel(writer, index=False, sheet_name="Variabilidade_verbo_sujeito")


    elif tipo_construcao == "n_adj":
        df_var_nome_sorted = df_var_nome.sort_values(by="variabilidade_nome", ascending=False)
        df_var_adj_sorted = df_var_adj.sort_values(by="variabilidade_adjetivo", ascending=False)

        df_var_nome_sorted.to_excel(writer, index=False, sheet_name="Variabilidade_nome")
        df_var_adj_sorted.to_excel(writer, index=False, sheet_name="Variabilidade_adjetivo")

    elif tipo_construcao == "adj_n":
        df_var_sorted = df_var.sort_values(by="variabilidade_semântica", ascending=False)
        df_var_sorted.to_excel(writer, index=False, sheet_name="Variabilidade")

print(f"\n📁 Ficheiro exportado com sucesso: {output_path}")

